## Homework

In [1]:
import bm_preproc

In [60]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    alignments = 0
    comparisons = 0
    while i < len(t) - len(p) + 1:
        alignments += 1
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences, alignments, comparisons


#### Question 1
Boyer-Moore: How many alignments are skipped by the bad character rule for this alignment?

Note: the number of skips is one less than the number of positions P shifts by.  That is, if the pattern shifts by 2 positions, that's 1 alignment skipped.

Also note: the question is asking only about the alignment shown.  Do not consider any other alignments of P to T in your answer.

T: GGCTATAATGCGTA

P:  TAATAAA

In [5]:
T = 'GGCTATAATGCGTA'
P = 'TAATAAA'
P_bm = BoyerMoore(P)
skip = P_bm.bad_character_rule(4, 'G')
skipped_alignments = skip - 1
print(skipped_alignments)

4


#### Question 2

Boyer-Moore: How many alignments are skipped by the good suffix rule in this scenario?

T: GGCTATAATGCGTA

P:  TAATTAA

In [6]:
skip = P_bm.good_suffix_rule(4)
skipped_alignments = skip - 1
print(skipped_alignments)


0


## Project

#### Question 1

How many alignments does the naive exact matching algorithm try when matching the string 
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [7]:
def readGenome(filename):
    genome=''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [56]:
def naive(p, t):
    occurrences = []
    alignments = 0
    comparisons = 0
    for i in range(len(t) - len(p) + 1):
        alignments += 1
        match = True
        for j in range(len(p)):
            comparisons += 1
            if t[i+j] != p[j]:
                match = False
                break
        if match:
            occurrences.append(i)
    return occurrences, alignments, comparisons

In [9]:
p = 'GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [10]:
num_alignments = len(genome) - len(p) + 1
print("Naive alignments:", num_alignments)

Naive alignments: 799954


In [74]:
# From another user
occurrences, alignments, comparisons = naive('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', genome)
print(f"Total alignments: {alignments}")

Total alignments: 799954


#### Question 2

How many character comparisons does the naive exact matching algorithm try when matching the string 
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [57]:
print("Comparisons done:", comparisons)

Comparisons done: 984143


#### Question 3

How many alignments does Boyer-Moore try when matching the string 
GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG (derived from human Alu sequences) to the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [61]:
# occurrences, alignments, comparisons = boyer_moore('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG', BoyerMoore('GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'), genome)
p_bm = BoyerMoore(p)
occurrences, alignments, comparisons = boyer_moore(p, p_bm, genome)
print(alignments)

127974


#### Question 4

Index-assisted approximate matching. In practicals, we built a Python class called Index implementing an ordered-list version of the k-mer index. The 
Index class is copied below.

In [62]:
def approximate_match_index(p, t, k):
    segment_length = round(len(p) // (k+1))
    hits = 0
    all_matches = set()
    index = Index(t, 8) # built on 8-mers
    for i in range(k+1):
        start = i * segment_length
        end = min((i+1) * segment_length, len(p))
        matches = index.query(p[start:end])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, start):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            for j in range(end, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            if mismatches <= k:
                all_matches.add(text_offset)
    return list(all_matches), hits

We also implemented the pigeonhole principle using Boyer-Moore as our exact matching algorithm.

Implement the pigeonhole principle using Index to find exact matches for the partitions. Assume P always has length 24, and that we are looking for approximate matches with up to 2 mismatches (substitutions). We will use an 8-mer index.

Write a function that, given a length-24 pattern P and given an Index object built on 8-mers, finds all approximate occurrences of P within T with up to 2 mismatches. Insertions and deletions are not allowed. Don't consider any reverse complements.

How many times does the string GGCGCGGTGGCTCACGCCTGTAAT, which is derived from a human Alu sequence, occur with up to 2 substitutions in the excerpt of human chromosome 1?  (Don't consider reverse complements here.)

Hint 1: Multiple index hits might direct you to the same match multiple times, but be careful not to count a match more than once.

Hint 2: You can check your work by comparing the output of your new function to that of the naive_2mm function implemented in the previous module.

In [64]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 2
ans, hits = approximate_match_index(p, t, k)
print("occurrences: " + str(len(ans)))

occurrences: 19


#### Question 5

Using the instructions given in Question 4, how many total index hits are there when searching for occurrences of 
GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?

  (Don't consider reverse complements.)

Hint: You should be able to use the boyer_moore function (or the slower naive function) to double-check your answer.

In [65]:
print("index hits: " + str(hits))

index hits: 90


#### Question 6
Let's examine whether there is a benefit to using an index built using subsequences of T rather than substrings, as we discussed in the "Variations on k-mer indexes" video.  We'll consider subsequences involving every N characters.  For example, if we split ATATAT into two substring partitions, we would get partitions ATA (the first half) and TAT (second half).  But if we split ATATAT into two  subsequences  by taking every other character, we would get AAA(first, third and fifth characters) and TTT (second, fourth and sixth).

Another way to visualize this is using numbers to show how each character of P is allocated to a partition.  Splitting a length-6 pattern into two substrings could be represented as 111222 and splitting into two subsequences of every other character could be represented as 121212

The following class SubseqIndex is a more general implementation of Index that additionally handles subsequences. It only considers subsequences that take every Nth character:

In [40]:
import bisect
   
class SubseqIndex(object):
    """ Holds a subsequence index for a text T """
    
    def __init__(self, t, k, ival):
        """ Create index from all subsequences consisting of k characters
            spaced ival positions apart.  E.g., SubseqIndex("ATAT", 2, 2)
            extracts ("AA", 0) and ("TT", 1). """
        self.k = k  # num characters per subsequence extracted
        self.ival = ival  # space between them; 1=adjacent, 2=every other, etc
        self.index = []
        self.span = 1 + ival * (k - 1)
        for i in range(len(t) - self.span + 1):  # for each subseq
            self.index.append((t[i:i+self.span:ival], i))  # add (subseq, offset)
        self.index.sort()  # alphabetize by subseq
    
    def query(self, p):
        """ Return index hits for first subseq of p """
        subseq = p[:self.span:self.ival]  # query with first subseq
        i = bisect.bisect_left(self.index, (subseq, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != subseq:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [66]:
# For example, if we do:
ind = SubseqIndex('ATATAT', 3, 2)
print(ind.index)

[('AAA', 0), ('TTT', 1)]


In [67]:
# And if we query this index:
p = 'TTATAT'
print(ind.query(p[0:]))

[]


In [68]:
# because the subsequence TAA is not in the index. But if we query with the second subsequence:
print(ind.query(p[1:]))

[1]


Write a function that, given a length-24 pattern P and given a SubseqIndex object built with k = 8 and ival = 3, finds all approximate occurrences of P within T with up to 2 mismatches.

When using this function, how many total index hits are there when searching for GGCGCGGTGGCTCACGCCTGTAAT with up to 2 substitutions in the excerpt of human chromosome 1?  (Again, don't consider reverse complements.)

Hint: See this notebook for a few examples you can use to test your function.

In [70]:
def approximate_match_subseq_index(p, t, k):
    segment_length = round(len(p) // (k+1))
    hits = 0
    all_matches = set()
    index = SubseqIndex(t, 8, 3) # built on 8-mers and subsequence intervals of 3
    for start in range(k+1):
        matches = index.query(p[start:])
        hits += len(matches)
        for m in matches:
            text_offset = m - start
            if text_offset < 0 or (text_offset + len(p)) > len(t):
                continue
            mismatches = 0
            for j in range(0, len(p)):
                if not p[j] == t[text_offset + j]:
                    mismatches += 1
                    if mismatches > k:
                        break
            if mismatches <= k:
                all_matches.add(text_offset)
    return list(all_matches), hits

In [71]:
t = genome
p = 'GGCGCGGTGGCTCACGCCTGTAAT'
k = 2
_, hits = approximate_match_subseq_index(p, t, k)
print(hits)

79
